# Vérification du codage de la distance au bord le plus proche

In [ ]:
from trustutils import run
import os

run.introduction("C. Reiss","29/04/2022")

run.TRUST_parameters()

### Description: 

This form tests the functions that allow to fill the wall distance tables and post-process it. To determine this distance, the algorithm loads all the coordinates on the wall faces, vertices and edges. It exchanges this information between the processors. It uses the `findClosestTupleId` in MEDCoupling for an efficient search. This method is not exact on deformed meshes or with tetrahedrons because the center projection of a volume element on a face is not exactly on its center of gravity or a vertex. To ensure accurate results for edges, a loop must be executed at the end of the iteration.

In [ ]:
maillage = {
    2: {"PolyMAC": " ", "PolyMAC_P0": " ", "PolyMAC_P0P1NC": " ", "VDF": " ", "VEFPreP1B": " trianguler_h dom "},
    3: {
        "PolyMAC": " polyedriser dom ",
        "PolyMAC_P0": " polyedriser dom ",
        "PolyMAC_P0P1NC": " polyedriser dom ",
        "VDF": "  ",
        "VEFPreP1B": " tetraedriser_homogene dom ",
    },
}

In [ ]:
force_recalculation = True
number_of_partitions = 1
build_dir = run.BUILD_DIRECTORY

if (force_recalculation) :
    run.reset()
    run.initBuildDirectory()

    for dim, meshes in maillage.items():
        for scheme, mesh in meshes.items():
            name = f"{scheme}_{dim}D"
            sub_dict = {"schema" : scheme, "maillage" : mesh}
            case = run.addCaseFromTemplate(f"jdd_{dim}D.data", name, sub_dict, targetData=f"{name}.data", nbProcs=number_of_partitions)
            if number_of_partitions > 1:
                case.partition()

    case = run.addCaseFromTemplate("jdd_triangle.data", "TRIANGLE", {}, nbProcs=number_of_partitions)
    if number_of_partitions > 1:
        case.partition()

    run.printCases()
    run.runCases()

## Display the wall distance field in 2D on a complicated geometry

In [ ]:
from trustutils import visit
from trustutils.jupyter import plot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

par = ""
if (number_of_partitions > 1):
    par = "PAR_"

for scheme, mesh in maillage[2].items():
    name = f"{scheme}_2D"
    fig=visit.showField(f"{name}/{par}{name}.lata","Pseudocolor","DISTANCE_PAROI_GLOBALE_ELEM_dom",iteration=0)



## Comparison of probes in 2D and 3D against an analytical solution

The distance from the probes is plotted for every scheme in 2D and 3D. The name of the probe is related to its position in the mesh. We use less points in 3D to keep a reasonable computation time. The differences in VEF are probably due to the gravity centers shift resulting from the tetrahedron or triangular meshes.

In [ ]:
fig = plt.figure(figsize=(12, 8))

plt.subplots_adjust(hspace= 0.3)

dic_subplots = { "NORD" : fig.add_subplot(2, 6, (1,2)) ,
                 "OUEST": fig.add_subplot(2, 6, (3, 4)),
                 "SUD1" : fig.add_subplot(2, 6, (5, 6)),
                 "SUD2" : fig.add_subplot(2, 6, (7,9)) ,
                 "EST"  : fig.add_subplot(2, 6, (10,12)) 
                }

absc = { "NORD" : np.linspace(-.975, .975, 40) ,
         "OUEST": np.linspace(-1.9, -1.1, 5),
         "SUD1" : np.linspace(-1.9, -1.1, 5),
         "SUD2" : np.linspace(.025, 4.975, 100),
         "EST"  : np.linspace(2**0.5*2.95, 0, 60)
        }

dict_marker = { "PolyMAC" : ":", 
               "PolyMAC_P0": ":", 
               "PolyMAC_P0P1NC": ":", 
               "VDF" : "--", 
               "VEFPreP1B" : "-" }
dict_linesize = { "PolyMAC" : 4, 
                 "PolyMAC_P0": 4, 
                 "PolyMAC_P0P1NC": 4, 
                 "VDF" : 3, 
                 "VEFPreP1B" : 2 }

tab_color = [ 0, 0, "blue", "red"]

xe_loc = 4.975-absc["EST"]/2**.5
ye_loc = -.975+absc["EST"]/2**.5

ana = { "NORD" : 0.5 - absc["NORD"] + absc["NORD"] ,
         "OUEST": .5-np.abs(absc["OUEST"]+1.5),
         "SUD1" : np.abs(absc["SUD1"]+1),
         "SUD2" : np.minimum( 5-absc["SUD2"] , ( np.maximum(absc["SUD2"]-2, 0)**2 + 0.9**2)**.5 ),
         "EST"  : np.minimum( ((xe_loc-5)**2+(ye_loc+1)**2)**.5 , xe_loc-2)
        }

dic_subplots["NORD"].set_ylim(.4, .8)

for loc in dic_subplots.keys():
    for d, meshes in maillage.items():
        for s, m in meshes.items():
            name = f"{s}_{d}D"
            par = ""
            if (number_of_partitions > 1): par = "PAR_"
            dist = np.array(plot.loadText(f"{name}/{par}{name}_{loc}.son")[1::])
            dic_subplots[loc].plot(absc[loc], dist, dict_marker[s], linewidth = dict_linesize[s], color = tab_color[d] , label = name)
    dic_subplots[loc].plot(absc[loc], ana[loc], color = "k", label = "Analytique")
    dic_subplots[loc].legend(fontsize = 6, ncol = 2)
    dic_subplots[loc].set_xlabel("x-position")
    dic_subplots[loc].set_ylabel("distance from wall")
    dic_subplots[loc].set_title(loc)

plt.tight_layout()
#plt.savefig("Distance_paroi_tous_schemas.pdf")



## Wall distance on tetrahedron near the wall

We compare the wall distance on a 2D triangular mesh with theory. We use a probe on the cells closest to the wall at Y=0.5. We can see that the exact solution is found in the wall element thanks to an additional loop but not on the next one as the computed distance is greater.

In [ ]:
fig=visit.showField(f"TRIANGLE/{par}jdd_triangle.lata", "Pseudocolor", "DISTANCE_PAROI_GLOBALE_ELEM_dom")

In [ ]:
d = np.array(plot.loadText(f"TRIANGLE/{par}jdd_triangle_DIST.son")[1::])
x = np.linspace(0.5, 9.5, 20)

d_min = np.linspace(0.333333, 0.333333, 20)
d_max = np.linspace(0.666666, 0.666666, 20)

fig, ax = plt.subplots()

ax.plot(x, d_min, 'b', linewidth = 2, label = 'Possible low value')
ax.plot(x, d_max, 'g', linewidth = 2, label = 'Possible high value')
ax.plot(x,     d, 'or', label = 'trust output')
plt.xlabel("position on the probe")
plt.ylabel("distance to the edge")
plt.ylim(0, 1)
ax.legend(bbox_to_anchor=(1.5, 0.75))

plt.show()